Есть датфрейм с authors, text, annotation и title. Текст предобработан, стоп-слова удалены. У одной статьи может быть несколько авторов, тогда они записаны в одной ячейке authors и разделены запятой. 

Функции:
- для вычисления всех соавторов одного человека по ФИО, они могут быть записаны в разных ячейках датафрейма. 

- функция для векторизации текста.  

- функция для вычисления коэффициента схожести.

- функция для рекомендации новых соавторов. 

В функцию для рекомендации новых соавторов подаётся на вход ФИО автора. На выходе ФИО автора, названия всех статей в которых он участвовал, список его соавторов, список трёх рекомендуемых новых авторов/коллективов авторов, названия их статей и коэффициент близости. 

Ограничения:

- Нельзя рекомендовать автора самому себе 
- Нельзя рекомендовать автору его текущих соавторов. 

Особенности:

Необходимо предусмотреть ситуацию когда рекомендуется несколько человек, написавших одну и ту же статью, при совпадении коэффициента близости у новых соавторов необходимо объединять их ФИО через запятую, писать их общую статью один раз и на 2 и 3 месте писать новых рекомендуемых авторов. При совпадении 2 и 3 рекомендации также следует их объединять и на третье место выводить следующих по коэффициенту схожести коллектив авторов/автора.

In [5]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import numpy as np
import pprint
from gensim.models import Word2Vec
from sentence_transformers import SentenceTransformer
import os

In [2]:
df = pd.read_excel('date_it_lemm.xlsx')

In [39]:
#df['authors'] = df['authors'].str.split(',') #авторов через запятую разделяем

In [17]:
def get_coauthors(df, author_name):
    coauthors = set()
    for authors_list in df['authors'].str.split(', '):
        if author_name in authors_list:
            coauthors.update(authors_list)
    coauthors.discard(author_name)  # удаляем самого автора
    return list(coauthors)

def calculate_similarity(tfidf_matrix, target_indices, all_indices):
    similarities = cosine_similarity(tfidf_matrix[target_indices], tfidf_matrix[all_indices])
    return similarities.max(axis=0)

In [19]:
def load_sentence_transformer_model(model_name='paraphrase-multilingual-MiniLM-L12-v2', save_path='sentence_transformer_model'):
    # Если модель уже сохранена, загружаем её
    if os.path.exists(save_path):
        print("Загрузка модели Sentence Transformer с диска...")
        return SentenceTransformer(save_path)
    else:
        # Иначе загружаем модель из библиотеки и сохраняем
        print("Загрузка модели Sentence Transformer из библиотеки...")
        model = SentenceTransformer(model_name)
        model.save(save_path)
        return model

In [21]:
def vectorize_texts_with_sentence_transformer(df, text_column='text', model_path='sentence_transformer_model'):
    # Загружаем модель Sentence Transformer
    model = load_sentence_transformer_model(save_path=model_path)
    
    # Векторизация текстов
    print("Векторизация текстов...")
    texts = df[text_column].tolist()
    vectors = model.encode(texts)  # Векторизация всех текстов
    
    return vectors, model

In [23]:
def recommend_new_coauthors(df, author_name, model_path='sentence_transformer_model'):
    target_mask = df['authors'].str.contains(author_name, regex=False)  # Данные автора
    target_indices = df[target_mask].index.tolist()
    
    if not target_indices:
        return {"error": "Автор не найден"}
    
    current_coauthors = get_coauthors(df, author_name)  # Текущие соавторы
    
    # Векторизация текстов с помощью Sentence Transformer
    tfidf_matrix, _ = vectorize_texts_with_sentence_transformer(df, text_column='text', model_path=model_path)
    
    all_indices = df.index.tolist()
    similarities = calculate_similarity(tfidf_matrix, target_indices, all_indices)  # Схожести

    results = pd.DataFrame({
        'index': all_indices,
        'similarity': similarities,
        'authors': df['authors'],
        'title': df['title']
    })
    
    results = results[
        (results['similarity'] < 1.0) &  # Исключаем статьи автора
        (~results['authors'].isin([author_name]))  # Исключаем соавторов
    ]
    
    # Группировка по коэффициенту схожести
    grouped = results.groupby('similarity').agg({
        'authors': lambda x: ', '.join(sorted(set(', '.join(x).split(', ')))),
        'title': lambda x: ', '.join(sorted(set(x)))
    }).reset_index().sort_values('similarity', ascending=False)
    
    recommendations = []
    used_authors = set(current_coauthors + [author_name])  # Формируем рекомендации
    
    for _, row in grouped.iterrows():
        candidates = [a.strip() for a in row['authors'].split(',') if a.strip() not in used_authors]
        if candidates:
            unique_candidates = []
            for candidate in candidates:
                if candidate not in used_authors:
                    unique_candidates.append(candidate)
                    used_authors.add(candidate)
            
            if unique_candidates:
                recommendations.append({
                    'authors': ', '.join(unique_candidates),
                    'titles': row['title'],
                    'similarity': round(row['similarity'], 2)
                })
        
        if len(recommendations) >= 3:
            break
    
    # Итоговый вывод
    output = {
        "Автор": author_name,
        "Текущие соавторы": current_coauthors,
        "Статьи": df[target_mask]['title'].tolist(),
        "Рекомендуемые соавторы": recommendations[:3]
    }
    
    # Сохраняем результаты в файлы
    save_results_to_txt(author_name, current_coauthors, output["Статьи"], output["Рекомендуемые соавторы"], df)
    
    return output

In [13]:
# Загружаем модель Sentence Transformer (она автоматически сохранится на диск)
model = load_sentence_transformer_model(save_path='sentence_transformer_model')

Загрузка модели Sentence Transformer из библиотеки...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

C:\Users\ksurashanti\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ksurashanti\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [27]:
def save_results_to_txt(author_name, current_coauthors, articles, recommendations, df):
    # Создаем папку для сохранения файлов (если её нет)
    import os
    if not os.path.exists("output"):
        os.makedirs("output")

    # Файл 1: Информация об авторе, его соавторах и нулевой статье
    with open(f"output/{author_name}_info.txt", "w", encoding="utf-8") as f:
        f.write(f"Автор: {author_name}\n")
        f.write(f"Соавторы: {', '.join(current_coauthors)}\n\n")
        
        # Проверяем, есть ли статьи у автора
        if articles:
            try:
                first_article = df[df['title'] == articles[0]].iloc[0]
                f.write(f"Заголовок: {first_article['title']}\n")
                f.write(f"Аннотация: {first_article['annotation']}\n")
                f.write(f"Текст: {first_article['text']}\n")
            except IndexError:
                f.write("Нулевая статья не найдена.\n")
        else:
            f.write("Статьи автора не найдены.\n")

    # Файлы 2-4: Рекомендуемые статьи
    for i, rec in enumerate(recommendations[:3], start=1):
        with open(f"output/{author_name}_recommendation_{i}.txt", "w", encoding="utf-8") as f:
            f.write(f"Коэффициент схожести: {rec['similarity']}\n")
            f.write(f"Рекомендуемые авторы: {rec['authors']}\n\n")
            
            titles = rec['titles'].split(', ')
            for title in titles:
                try:
                    article = df[df['title'] == title].iloc[0]
                    f.write(f"Заголовок: {article['title']}\n")
                    f.write(f"Аннотация: {article['annotation']}\n")
                    f.write(f"Текст: {article['text']}\n\n")
                except IndexError:
                    f.write(f"Статья с заголовком '{title}' не найдена.\n\n")

In [29]:
import pprint

# Пример вызова функции
rec_list = recommend_new_coauthors(df, 'Большаков А. О.', model_path='sentence_transformer_model')
pprint.pprint(rec_list)

Загрузка модели Sentence Transformer с диска...
Векторизация текстов...
{'Автор': 'Большаков А. О.',
 'Рекомендуемые соавторы': [{'authors': 'Колесникова Ольга Валерьевна, Лелюхин '
                                        'Владимир Егорович, Осипова Марина '
                                        'Анатольевна, Цициашвилли Гурами '
                                        'Шалвович',
                             'similarity': 0.94,
                             'titles': 'Формальная интерпретация задачи поиска '
                                       'технологических баз и синтеза '
                                       'последовательности обработки '
                                       'поверхностей детали'},
                            {'authors': 'Жабин Я. О., Леонтьев А. C.',
                             'similarity': 0.94,
                             'titles': 'АНАЛИЗ ПРОЦЕССА ПОИСКА ИГРОКОВ И '
                                       'ФОРМИРОВАНИЯ КОМАНД ДЛЯ '
                 

Вывод: здесь совершенно не подходит статья про обработку поверхностей детали, а она самая рекомендуемая